# Teleoperación

En este ejemplo controlaremos remótamente el robot con un mando conectado a nuestro ordenador.

### Controlador para el mando

Para poder dirigir nuestro robot, crearemos una instancia del _widget_``Controller``. Esta clase nos proporciona una interfaz para un mando, con el que podremos pilotar nuestro robot. 

El _widget_ ``Controller``  toma un índice (``index``) como parámetro, el cuál especifica que mando de los que están conectados se empleará como entrada del mismo. Esto es útil cuando tenemos varios controladores conectados de forma simutanea, o para algunos controladores que puedan aparecer como varios controladores en el sistema.

Para conocer el índice del controlador que queremos usar como entrada:

1. Acceder a [http://html5gamepad.com](http://html5gamepad.com).  
2. Encender y presionar alguno de los botones del mando hasta que la página lo reconozca.
3. Aparecerá un identificador (``index``) del controlador con el que se está interaccionando. Usar este índice como parámetro del _widget_.

A continuación, se creará y se visualizará la interfaz del mando:

In [ ]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  #Reemplazar por indice del mando

display(controller)

Es posible que aparezca el mensaje ``Connect gamepad and press any button``. Esto es debido a que el mando aún no se ha registrado en el notebook. Presiona cualquier botón y verás como aparece una visualización del _widget_.

### Conectar el controlador del mando a los motores del robot

Una vez creada la instancia del controlador del mando, debemos contectarlos al control de nuestro robot. El control más sencillo sería poder modificar la velocidad de las ruedas del motor, usando el _joystick_ izquierdo para manejar el motor izquierdo y, de la misma forma, el derecho. 

Para ello usaremos la función  ``dlink``. A diferencia de la función ``link``, ``dlink`` crea un vínculo unidireccional, definiendo un atributo de origen y otro como destino. Además, se puede añadir una función de transformación del valor entre el origen y el destino. 

De forma intuitiva, mover un joystick hacía arriba llevaría a mover la rueda correspondiente hacia delante. Sin embargo, debido a como están dispuestos los ejes de los joysticks, los controles parecen invertidos. Por ello, emplearemos una función de transformación que niegue el valor.

> ATENCIÓN: Ejecutar este bloque de código hará que el robot se mueva si interaccionas con el mando. Comprobar primero que el robot tiene espacio suficiente.

In [ ]:
from jetbot import Robot
import traitlets

robot = Robot()

def invert_movement(x):
    return -x

left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=invert_movement)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=invert_movement)

Para evitar tener que definir una función tan pequeña y que sólo se usará esta vez, podemos usar en su lugar una expresión _lambda_. Estas funciones pueden simplificar el código, y se encuentran comunmente. Para el ejemplo anterior, el código resultaría:

In [ ]:
# left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
# right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)

> ATENCIÓN: No ejecutar este fragmento de código sin desvincular antes las acciones para evitar ducplicados. 

### Conexión con la cámara

1) Crear un _widget_ para la visualización:

Usaremos un _widget_ de tipo ``Image`` donde mostrar el _feed_ en directo de la cámara de nuestro robot. Tenemos que indicar la resolución de la misma dando valores a los parámetros de ``width`` (ancho) y ``height`` (alto). La fijaremos en 300x300 píxles para que no ocupe demasiado espacio. Hay que tener en cuenta, no obstante, que la resolución del _widget_ sólo afecta a la visualización por la parte del navegador, no a la resolución con la que imagen es capturada por la cámara de forma nativa.

In [ ]:
image = widgets.Image(format='jpeg', width=300, height=300)

display(image)

La imagen aparece vacía ya que, de momento, sólo hemos creado el _widget_ donde visualizar la imagen, pero aún no le hemos dado un valor. 

2) Crear una instancia de la cámara

A continuación, debemos inicializar la cámara. Para ello, crearemos una instancia de la clase ``Camera`` y conectaremos el atributo del valor (``value``) de la cámara al del _widget_ de la imagen de visualización.

Para instanciar la cámara se debe llamar al método ``instance`` que devolverá una nueva instancia de la cámara si esta no existe. Si ya exisitía, devolverá una referencia a la cámara existente.

> INFORMACIÓN: Esta forma de instanciar un objeto se da con el patrón de diseño del _singleton_, en el que se restringe la creación de objetos pertenecientes a una clase a un único objeto. 

In [ ]:
from jetbot import Camera

camera = Camera.instance()

3) Conectar la cámara al _widget_ de la imagen: cambio de formato

Nuestra cámara captura imágenes en formato BGR8, mientras que nuestro _widget_ de imagen sólo acepta valores en el formato comprimido *JPEG*. Por lo tanto, será necesario transformar de un tipo de formato a otro. Para ello, usaremos la función``bgr8_to_jpeg``, incluida en este tutorial, como función de transformación en el enlace. 

> INFORMACIÓN: El formato BGR8 indica que la imagen está formada por tres canales de color, BGR (de _blue_,_green_ y _red_), con una profundidad de 8 bits cada uno. En este caso, 8 bits para el valor de cada uno de los componentes, por lo que tenemos valores que van de 0 a 255.

In [ ]:
from jetbot import bgr8_to_jpeg

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

Tras este paso, la salida de la cámara debería estar conectada al widget de la imagen, y se debería transmitir la señal en directo.

> RECUERDA: Puedes hacer click derecho en salida del _widget_ y seleccionar ``Create New View for Output`` para mostrar el _widget_ en una nueva ventana. 

### Paro de emergencia si se pierde la conexión con la señal

¿Qué ocurre si el robot se desconecta de la red WiFi? El robot permanecería en movimiento, intentando comunicar imágenes de vídeo y comandos para el control de los motores. Para evitar este comportamiento, estableceremos un sistema de seguridad que pare el robot y desconecte la cámara y motores cuando se desconecte de la red WiFi. Para ello, usaremos la clase ``Heartbeat`` empleada en el tutorial anterior. 

In [ ]:
from jetbot import Heartbeat

def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)

# Se conecta la función de seguridad en el caso de que el estado del Heartbeat cambie
heartbeat.observe(handle_heartbeat_status, names='status')

> Si la conexión con el robot se pierde, este parará. Para recuperar el control del robot, una vez este vuelva a estar conectado, será necesario realizar de nuevo las conexiones entre los componentes. Para ello, podemos ejecutar el siguiente bloque de comunicación. 

In [ ]:
# only call this if your robot links were unlinked, otherwise we'll have redundant links which will double
# the commands transfered

left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

### Guarda capturas de pantalla con un botón del mando

Nos interesa poder guardar imágenes tomadas por nuestro robot para pasos posteriores. Para ello, haremos que un botón del mando sirva para poder salvar una captura de la imagen actual transmitada por el robot. 

Crearemos un directorio en esta misma carpeta llamado ``capturas/``, en la que iremos guardando las imágenes obtenidas.

In [2]:
import subprocess

subprocess.call(['mkdir', '-p', 'capturas'])

Cada archivo tendrá que tener un nombre único, por lo que emplearemos el paquete de python ``uuid`` (con el que se pueden generar identificadores únicos). Usaremos el identificador ``uuid1`` que también codifica la fecha y la dirección MAC, por si la necesitásemos más adelante. En el siguiente bloque definiremos la función que, al pulsar el botón, se ejecutará para guardar la imagen y enseñarla en un _widget_ de imagen.

In [ ]:
import uuid

snapshot_image = widgets.Image(format='jpeg', width=300, height=300)

def save_snapshot(change):
    # Guardar una captura de la cámara cuando se pulse el botón asignado
    if change['new']:
        # Creacion de la ruta al archivo
        file_path = 'capturas/' + str(uuid.uuid1()) + '.jpg'
        
        # Escribir el contenido de la imagen en el archivo 
        # (usaremos el valor del widget porque ya está convertido a JPEG)
        with open(file_path, 'wb') as f:
            f.write(image.value)
            
        # Mostrar en el widget
        snapshot_image.value = image.value

El botón a utilizar podrá ser cualquiera de lo accionadores digitales del mando, simplemente hay que buscar el índice correspondiente para el botón escogido. Por ejemplo, acorde al mapeo del mando de la XBOX, el botón _RB_ (gatillo superior derecho) tiene el valor 5. 

In [ ]:
controller.buttons[5].observe(save_snapshot, names='value')

display(widgets.HBox([image, snapshot_image]))
display(controller)